In [4]:
import cv2
from skimage import feature
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import os
import joblib
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


In [58]:
# Tải bộ phát hiện khuôn mặt Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Mở webcam
cap = cv2.VideoCapture(0)

capture_flag = False
cropped_images = []
while True:
    # Đọc một khung hình từ webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi hình ảnh từ BGR (OpenCV mặc định) sang grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Vẽ bounding boxes xung quanh các khuôn mặt
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Hiển thị khung hình với bounding boxes
    cv2.imshow('Face Detection', frame[:, ::-1])

    # Nhấn Space để chụp
    if cv2.waitKey(1) & 0xFF == ord(' '):
        capture_flag = True
    
    if capture_flag:
        if len(faces) == 0:
            print("Không có khuôn mặt!")
            capture_flag = False
            pass
        else:
            for (x, y, w, h) in faces:
                cropped_images.append(frame[y:y+h, x:x+w])
            print(faces)
            break

# Giải phóng nguồn tài nguyên
cap.release()
cv2.destroyAllWindows()

max_face = []
for img in cropped_images:
    if len(img) > len(max_face):
        max_face = img
cv2.imshow('face', max_face)

gray_face = cv2.cvtColor(max_face, cv2.COLOR_BGR2GRAY)
face_resized = cv2.resize(gray_face, (128, 128))
hog_features = feature.hog(face_resized).reshape(1, -1)
print(RF.predict(hog_features))
print(RF.predict_proba(hog_features))        

cv2.waitKey(0)
cv2.destroyAllWindows()

[[205 129 257 257]]
[0]
[[0.54434127 0.45565873]]


In [47]:
cap.release()
cv2.destroyAllWindows()

In [44]:
# LẤY DATA TỪ CAM

# Tải bộ phát hiện khuôn mặt Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Mở webcam
cap = cv2.VideoCapture(0)
count_frame = 0
capture_flag = False
cropped_images = []
i = 152
while True:
    # Đọc một khung hình từ webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi hình ảnh từ BGR (OpenCV mặc định) sang grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Vẽ bounding boxes xung quanh các khuôn mặt
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        if count_frame % 10 == 0:
            cv2.imwrite(f'./data/unknown/unknown{i}.jpg', frame[y:y+h, x:x+w])
            i += 1
        
    count_frame += 1
    # Hiển thị khung hình với bounding boxes
    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

    # nghi
    if i == 1000:
        break

# Giải phóng nguồn tài nguyên
cap.release()
cv2.destroyAllWindows()


<p1> TẠO DATA TRAIN - VALIDATE - TEST

In [2]:
def create_hog_vector(dir, hog_vector, target, target_value):
    for item in os.listdir(dir):
        img_dir = os.path.join(dir, item)
        img = cv2.imread(img_dir)
        img_resize = cv2.resize(img, (128, 128)) # resize ảnh bằng nhau
        gray_face = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
        features = feature.hog(gray_face)
        hog_vector.append(np.ravel(features))
        target.append(target_value)

# Bắt đầu duyệt
hog_vector = []
target = []

Nhat_dir = './data/Nhat'
Truong_dir = './data/Truong'

create_hog_vector(Nhat_dir, hog_vector, target, 0)
create_hog_vector(Truong_dir, hog_vector, target, 1)

<p1> TRAIN TEST PLIT

In [17]:
# Phân chia dữ liệu thành tập test và tập còn lại
X_train_val, X_test, y_train_val, y_test = train_test_split(hog_vector, target, test_size=0.2, random_state=42, stratify=target)

# Phân chia tập còn lại thành tập validation và tập train
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

<p> TEST UNKNOWN

In [39]:
hog_unknown = []
target_unknown = []
dir_unknown = './data/unknown'

create_hog_vector(dir_unknown, hog_unknown, target_unknown, 0)

def test_unknown_face(hog_vector):
    for item in hog_vector:
        print(RF.predict_proba(hog_features))

test_unknown_face(hog_unknown)

[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376 0.77217624]]
[[0.22782376

<p1> MLP

In [354]:
mlp = MLPClassifier(hidden_layer_sizes=(15,), max_iter=50, activation='relu', solver='adam', alpha=0.01, random_state=42)
mlp.fit(X_train, y_train)

C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.01, hidden_layer_sizes=(15,), max_iter=50,
              random_state=42)

In [355]:
y_probs = mlp.predict_proba(X_val)
result = []
for i, probs in enumerate(y_probs):
    if (((probs[0] > probs[1]) and (y_val[i] == 0)) or ((probs[0] < probs[1]) and (y_val[i] == 1))):
        result.append(1)
    else:
        result.append(0)
print(result.count(0))
print(result.count(1))

3
480


In [357]:
y_pred = mlp.predict(X_val)
y_pred_test = mlp.predict(X_test)
accuracy = accuracy_score(y_val, y_pred)
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Accuracy:", accuracy, accuracy_test)

Accuracy: 0.9937888198757764 1.0


<p1> SVM

In [294]:
from sklearn.svm import SVC

SVC = SVC(probability=True, kernel='rbf')
SVC.fit(X_train, y_train)

probabilities = SVC.predict_proba(X_val)
probabilities

array([[1.86696986e-02, 9.81330301e-01],
       [3.14642118e-07, 9.99999685e-01],
       [9.99569429e-01, 4.30570722e-04],
       ...,
       [9.79055606e-01, 2.09443935e-02],
       [6.55067982e-03, 9.93449320e-01],
       [4.16806062e-03, 9.95831939e-01]])

In [358]:
y_pred = SVC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9989658738366081
Precision: 0.9979381443298969
Recall: 1.0
F1 Score: 0.9989680082559339


<p1> RANDOM FOREST

In [18]:
param_dist = {'n_estimators' : randint(3, 20), 'max_depth' : randint(3, 6)}
random_search = RandomizedSearchCV(RandomForestClassifier(), param_dist, n_iter=100, cv=5, random_state=42)
random_search.fit(X_train, y_train)
RF = random_search.best_estimator_

In [9]:
probabilities = RF.predict_proba(X_test)
probabilities

array([[0.18390902, 0.81609098],
       [0.12329474, 0.87670526],
       [0.98819315, 0.01180685],
       ...,
       [0.95260926, 0.04739074],
       [0.98822482, 0.01177518],
       [0.08015789, 0.91984211]])

In [23]:
y_pred = RF.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [336]:
confusion_matrix(y_test, y_pred)

array([[477,   6],
       [  1, 483]], dtype=int64)

In [339]:
joblib.dump(RF, "CS231_model_RF.pkl")

['CS231_model_RF.pkl']